In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

from compression import Region, Comparator
from ipywidgets import interact

# TODO remove unused imports which are only needed in the library file

%matplotlib notebook

In [ ]:
# TODO set up stuff like a custom data directory and temp directory here
# pass them into the functions

DATA_DIR = os.path.join(os.path.expanduser("~"), "data", "fits")
colourmap = plt.cm.viridis


In [ ]:
regions = {
#     "GALFACTS N4 Q full": Region.from_fits(
#         os.path.join(DATA_DIR, "GALFACTS_N4_cube_Q_DQA3.3.1.fits")
#     ),

#     "GALFACTS N4 Q 512x512": Region.from_fits(
#         os.path.join(DATA_DIR, "GALFACTS_N4_cube_Q_DQA3.3.1.fits"), 
#         size=(512, 512),
#         centre=(768, 4600)
#     ),

#     "GALFACTS N4 I full": Region.from_fits(
#         os.path.join(DATA_DIR, "GALFACTS_N4_cube_I_DQA3.3.1.fits")
#     ),

#     "GALFACTS N4 I 512x512": Region.from_fits(
#         os.path.join(DATA_DIR, "GALFACTS_N4_cube_I_DQA3.3.1.fits"), 
#         size=(512, 512),
#         centre=(768, 4600)
#     ),

#     "DEEP full": Region.from_fits(
#         os.path.join(DATA_DIR, "DEEP_5_IQU_cube.fits")
#     ),

#     "DEEP 512x512": Region.from_fits(
#         os.path.join(DATA_DIR, "DEEP_5_IQU_cube.fits"),
#         size=(512, 512),
#         centre=(2048, 2048)
#     ),

#     "supermosaic full": Region.from_fits(
#         os.path.join(DATA_DIR, "supermosaic.10.fits")
#     ),

#     "supermosaic 512x512": Region.from_fits(
#         os.path.join(DATA_DIR, "supermosaic.10.fits"),
#         size=(512, 512),
#         centre=(900, 2000)
#     ),
    
    "DEEP cropped": Region.from_fits(
        os.path.join(DATA_DIR, "cropped_512_512_10_3_DEEP_5_IQU.fits"),
        stokes=0,
        channel=3
    ),
}

In [ ]:
comparator = Comparator.compare_algorithms(regions["DEEP cropped"], colourmap)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 10]

fig, axs = plt.subplots(nrows=2, ncols=2)

comparator.plot("size_fraction", "raw_error_mean", "Size (fraction)", 
                "Raw data error (fraction mean)", axs[0][0])
comparator.plot("size_fraction", "raw_error_max", "Size (fraction)", 
                "Raw data error (fraction max)", axs[0][1])
comparator.plot("size_fraction", "image_error_mean", "Size (fraction)", 
                "Image error (fraction mean)", axs[1][0])
comparator.plot("size_fraction", "image_error_max", "Size (fraction)", 
                "Image error (fraction max)", axs[1][1])



In [ ]:
interact(comparator.show_images, size=0.5);